# **🥈This Notebook now rank 15 in LB!**
+ I used `TruncatedSVD(n_components=128, random_state=42)` both on train and test data.
+ CatBoostRegressor is used. To save time， only trained on fold_0. Maybe more fold will help to impore score.
+ This notebook is based on [FABIEN CROM](https://www.kaggle.com/code/fabiencrom/msci-multiome-quickstart-w-sparse-matrices). Please Upvoted if help !
+ Version_1 is a quick submission and version_2_3 show the training process.

In [1]:
import os, gc, pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from colorama import Fore, Back, Style
from matplotlib.ticker import MaxNLocator

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, scale
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import Ridge, LinearRegression, Lasso
from sklearn.metrics import mean_squared_error

import scipy
import scipy.sparse

import gc
import pickle
import warnings
warnings.filterwarnings('ignore')

submission_name = "citeseq_svd256_wdo+catboost.csv"

In [2]:
def correlation_score(y_true, y_pred):
    """Scores the predictions according to the competition rules. 
    
    It is assumed that the predictions are not constant.
    
    Returns the average of each sample's Pearson correlation coefficient"""
    if type(y_true) == pd.DataFrame: y_true = y_true.values
    if type(y_pred) == pd.DataFrame: y_pred = y_pred.values
    if y_true.shape != y_pred.shape: raise ValueError("Shapes are different.")
    corrsum = 0
    for i in range(len(y_true)):
        corrsum += np.corrcoef(y_true[i], y_pred[i])[1, 0]
    return corrsum / len(y_true)


# Preprocessing and cross-validation

We first load all of the training input data for Multiome. It should take less than a minute.

In [3]:
%%time
train_inputs = scipy.sparse.load_npz("C:/Users/Owner/Documents/dev/open-problem/multimodal-single-cell-as-sparse-matrix/train_multi_inputs_values.sparse.npz")

Wall time: 18.4 s


In [4]:
train_inputs = train_inputs.astype('float16', copy=False)

## PCA / TruncatedSVD
It is not possible to directly apply PCA to a sparse matrix, because PCA has to first "center" the data, which destroys the sparsity. This is why we apply `TruncatedSVD` instead (which is pretty much "PCA without centering"). It might be better to normalize the data a bit more here, but we will keep it simple.

In [5]:
%%time
pca = TruncatedSVD(n_components=128, random_state=42)
train_inputs = pca.fit_transform(train_inputs)
print(pca.explained_variance_ratio_.sum())

0.010997659
Wall time: 6min 45s


In [6]:
%%time
train_targets = scipy.sparse.load_npz("C:/Users/Owner/Documents/dev/open-problem/multimodal-single-cell-as-sparse-matrix/train_multi_targets_values.sparse.npz")

Wall time: 8.29 s


In [7]:
%%time
pca2 = TruncatedSVD(n_components=128, random_state=42)
train_target = pca2.fit_transform(train_targets)
print(pca2.explained_variance_ratio_.sum())

0.11779099
Wall time: 2min 2s


In [8]:
def save(name, model):
    with open(name, 'wb') as f:
        pickle.dump(model, f)

In [9]:
save('pca.pkl', pca)
save('pca2.pkl', pca2)

In [12]:
from catboost import CatBoostRegressor
params = {'learning_rate': 0.1, 
          'depth': 7, 
          'l2_leaf_reg': 4, 
          'loss_function': 'MultiRMSE', 
          'eval_metric': 'MultiRMSE', 
          'task_type': 'CPU', 
          'iterations': 200,
          'od_type': 'Iter', 
          'boosting_type': 'Plain', 
          'bootstrap_type': 'Bayesian', 
          'allow_const_label': True, 
          'random_state': 1
         }
model = CatBoostRegressor(**params)

In [13]:
n = 1

In [14]:
np.random.seed(42)
all_row_indices = np.arange(train_inputs.shape[0])
np.random.shuffle(all_row_indices)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

index = 0
score = []

# model = Ridge(copy_X=False)
d = train_inputs.shape[0]//n
for i in range(0, n*d, d):
    print(f'start [{i}:{i+d}]')
    ind = all_row_indices[i:i+d]    
    for idx_tr, idx_va in kf.split(ind):
        X = train_inputs[ind]
        Y = train_target[ind] #.todense()
        Yva = train_targets[ind][idx_va]
        Xtr, Xva = X[idx_tr], X[idx_va]
        Ytr = Y[idx_tr]
        del X, Y
        gc.collect()
        print('Train...')
        model.fit(Xtr, Ytr)
        del Xtr, Ytr
        gc.collect()
        s = correlation_score(Yva.todense(), model.predict(Xva)@pca2.components_)
        score.append(s)
        print(index, s)
        del Xva, Yva
        gc.collect()
        pkl_filename = f"model{index:02d}.pkl"
        index += 1
        with open(pkl_filename, 'wb') as file:
            pickle.dump(model, file)
#         break
#     break
    gc.collect()

start [0:105942]
Train...
0:	learn: 74.4837434	total: 4.49s	remaining: 14m 53s
1:	learn: 72.3356197	total: 9.02s	remaining: 14m 53s
2:	learn: 70.4881264	total: 13.5s	remaining: 14m 46s
3:	learn: 68.7940866	total: 18.1s	remaining: 14m 44s
4:	learn: 67.3404237	total: 22.5s	remaining: 14m 37s
5:	learn: 65.9576748	total: 27.1s	remaining: 14m 34s
6:	learn: 64.7607699	total: 31.6s	remaining: 14m 32s
7:	learn: 63.7088646	total: 36.2s	remaining: 14m 29s
8:	learn: 62.7753458	total: 40.8s	remaining: 14m 25s
9:	learn: 61.9207222	total: 45.3s	remaining: 14m 20s
10:	learn: 61.2033426	total: 49.7s	remaining: 14m 14s
11:	learn: 60.5610450	total: 54.2s	remaining: 14m 8s
12:	learn: 59.9598913	total: 58.7s	remaining: 14m 4s
13:	learn: 59.4176549	total: 1m 3s	remaining: 13m 58s
14:	learn: 58.9458861	total: 1m 7s	remaining: 13m 53s
15:	learn: 58.5225956	total: 1m 11s	remaining: 13m 47s
16:	learn: 58.0944770	total: 1m 16s	remaining: 13m 42s
17:	learn: 57.7123285	total: 1m 20s	remaining: 13m 37s
18:	learn: 

In [15]:
del train_target, train_inputs, train_targets
gc.collect()

0

# Predicting

In [16]:
%%time
multi_test_x = scipy.sparse.load_npz("C:/Users/Owner/Documents/dev/open-problem/multimodal-single-cell-as-sparse-matrix/test_multi_inputs_values.sparse.npz")
multi_test_x = pca.transform(multi_test_x)

Wall time: 25.7 s


In [17]:
# test_sd = np.std(multi_test_x, axis=1).reshape(-1, 1)
# test_sd[test_sd == 0] = 1
# test_norm = (multi_test_x - np.mean(multi_test_x, axis=1).reshape(-1, 1)) / test_sd
# test_norm = test_norm.astype(np.float16)
# del multi_test_x
# gc.collect()

In [18]:
test_len = multi_test_x.shape[0]
d = test_len//n
x = []
for i in range(n):
    x.append(multi_test_x[i*d:i*d+d])
del multi_test_x
gc.collect()

55

In [19]:
index

5

In [20]:
preds = np.zeros((test_len, 23418), dtype='float16')
for i,xx in enumerate(x):
    for ind in range(index):
        print(ind, end=' ')
        with open(f'model{ind:02}.pkl', 'rb') as file:
            model = pickle.load(file)
        preds[i*d:i*d+d,:] += (model.predict(xx)@pca2.components_)/index
        gc.collect()
    print('')
    del xx
gc.collect()

0 1 2 3 4 


0

In [21]:
del x
gc.collect()

0

In [22]:
np.save('preds.npy', preds)

In [24]:
!pip install pyarrow fastparquet

You should consider upgrading via the 'C:\Users\Owner\pyenvs\dev39\Scripts\python.exe -m pip install --upgrade pip' command.


# Creating submission

We load the cells that will have to appear in submission.

In [25]:
%%time
# Read the table of rows and columns required for submission
eval_ids = pd.read_parquet("C:/Users/Owner/Documents/dev/open-problem/multimodal-single-cell-as-sparse-matrix/evaluation.parquet")
# Convert the string columns to more efficient categorical types
#eval_ids.cell_id = eval_ids.cell_id.apply(lambda s: int(s, base=16))
eval_ids.cell_id = eval_ids.cell_id.astype(pd.CategoricalDtype())
eval_ids.gene_id = eval_ids.gene_id.astype(pd.CategoricalDtype())

Wall time: 15.4 s


In [26]:
# Prepare an empty series which will be filled with predictions
submission = pd.Series(name='target',
                       index=pd.MultiIndex.from_frame(eval_ids), 
                       dtype=np.float32)
submission

row_id    cell_id       gene_id        
0         c2150f55becb  CD86              NaN
1         c2150f55becb  CD274             NaN
2         c2150f55becb  CD270             NaN
3         c2150f55becb  CD155             NaN
4         c2150f55becb  CD112             NaN
                                           ..
65744175  2c53aa67933d  ENSG00000134419   NaN
65744176  2c53aa67933d  ENSG00000186862   NaN
65744177  2c53aa67933d  ENSG00000170959   NaN
65744178  2c53aa67933d  ENSG00000107874   NaN
65744179  2c53aa67933d  ENSG00000166012   NaN
Name: target, Length: 65744180, dtype: float32

We load the `index`  and `columns` of the original dataframe, as we need them to make the submission.

In [27]:
%%time
y_columns = np.load("C:/Users/Owner/Documents/dev/open-problem/multimodal-single-cell-as-sparse-matrix/train_multi_targets_idxcol.npz",
                   allow_pickle=True)["columns"]

test_index = np.load("C:/Users/Owner/Documents/dev/open-problem/multimodal-single-cell-as-sparse-matrix/test_multi_inputs_idxcol.npz",
                    allow_pickle=True)["index"]

Wall time: 29 ms


We assign the predicted values to the correct row in the submission file.

In [28]:
cell_dict = dict((k,v) for v,k in enumerate(test_index)) 
assert len(cell_dict)  == len(test_index)

gene_dict = dict((k,v) for v,k in enumerate(y_columns))
assert len(gene_dict) == len(y_columns)

In [29]:
eval_ids_cell_num = eval_ids.cell_id.apply(lambda x:cell_dict.get(x, -1))
eval_ids_gene_num = eval_ids.gene_id.apply(lambda x:gene_dict.get(x, -1))

valid_multi_rows = (eval_ids_gene_num !=-1) & (eval_ids_cell_num!=-1)

In [30]:
submission.iloc[valid_multi_rows] = preds[eval_ids_cell_num[valid_multi_rows].to_numpy(),
eval_ids_gene_num[valid_multi_rows].to_numpy()]

In [31]:
del eval_ids_cell_num, eval_ids_gene_num, valid_multi_rows, eval_ids, test_index, y_columns
gc.collect()

48

In [32]:
submission

row_id    cell_id       gene_id        
0         c2150f55becb  CD86                    NaN
1         c2150f55becb  CD274                   NaN
2         c2150f55becb  CD270                   NaN
3         c2150f55becb  CD155                   NaN
4         c2150f55becb  CD112                   NaN
                                             ...   
65744175  2c53aa67933d  ENSG00000134419    6.109375
65744176  2c53aa67933d  ENSG00000186862    0.033661
65744177  2c53aa67933d  ENSG00000170959    0.045166
65744178  2c53aa67933d  ENSG00000107874    1.147461
65744179  2c53aa67933d  ENSG00000166012    5.160156
Name: target, Length: 65744180, dtype: float32

# Merging with CITEseq predictions

We use the CITEseq predictions from [this notebook](https://www.kaggle.com/code/vuonglam/lgbm-baseline-optuna-drop-constant-cite-task) by VuongLam.

In [33]:
submission.reset_index(drop=True, inplace=True)
submission.index.name = 'row_id'

In [34]:
cite_submission = pd.read_csv("C:/Users/Owner/Documents/dev/open-problem/citeseq/submission_svd256_wdo.csv")
cite_submission = cite_submission.set_index("row_id")
cite_submission = cite_submission["target"]

In [35]:
submission[submission.isnull()] = cite_submission[submission.isnull()]

In [36]:
submission

row_id
0           0.094605
1          -0.162362
2          -0.405332
3          -0.302582
4           1.114355
              ...   
65744175    6.109375
65744176    0.033661
65744177    0.045166
65744178    1.147461
65744179    5.160156
Name: target, Length: 65744180, dtype: float32

In [37]:
submission

row_id
0           0.094605
1          -0.162362
2          -0.405332
3          -0.302582
4           1.114355
              ...   
65744175    6.109375
65744176    0.033661
65744177    0.045166
65744178    1.147461
65744179    5.160156
Name: target, Length: 65744180, dtype: float32

In [38]:
submission.isnull().any()

False

In [39]:
submission.to_csv(submission_name)

In [40]:
!head submission.csv

'head' �́A�����R�}���h�܂��͊O���R�}���h�A
����\�ȃv���O�����܂��̓o�b�` �t�@�C���Ƃ��ĔF������Ă��܂���B


In [41]:
!head submission.csv

'head' �́A�����R�}���h�܂��͊O���R�}���h�A
����\�ȃv���O�����܂��̓o�b�` �t�@�C���Ƃ��ĔF������Ă��܂���B
